# 1_emulator
notebook for loading in a trained emulator, incuding the .json file with custom_object variables and data scaling values.\
Emulator .model and .json are stored in files in the repo.

In [3]:
#### misc
import pandas as pd
import numpy as np
import os
from pathlib import Path
import pickle
import time
from itertools import product
from scipy import constants
from scipy import stats
import random
import astropy.constants
from numpy.random import default_rng
import json

#### graphical
import matplotlib.pyplot as plt

#### ML
import sklearn
from sklearn.decomposition import PCA
import tensorflow as tf
import keras
from keras import layers

#### custom
from InversePCA import InversePCA
from WMSE import WMSE, WMSE_metric

##### poke gpu
os.environ["CUDA_VISIBLE_DEVICES"]="0"

physical_devices = tf.config.list_physical_devices("GPU") 

gpu0usage = tf.config.experimental.get_memory_info("GPU:0")["current"]

print("Current GPU usage:\n"
     + " - GPU0: " + str(gpu0usage) + "B\n")

2024-03-12 14:24:08.918994: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 14:24:08.919026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 14:24:08.919887: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-12 14:24:08.924650: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 14:24:09.503579: W tensorflow/compiler/tf2

Current GPU usage:
 - GPU0: 0B



2024-03-12 14:24:10.025292: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17862 MB memory:  -> device: 0, name: NVIDIA RTX A4500, pci bus id: 0000:41:00.0, compute capability: 8.6


# 1) Specify emulator name
if file structure has been organised properly, just the name should find the emulator and unpack the relevant .json and .model files

In [1]:
pitchfork_name = "nu15-25_emulator"

# 2) Unpack .json file
Unpack .json, print architecture and emulator parameter ranges

In [25]:
json_file_path = pitchfork_name+"/"+pitchfork_name+".json"

with open(json_file_path, 'r') as fp:
     pitchfork_dict = json.loads(fp.read())

pca_comps = np.array(pitchfork_dict["custom_objects"]["inverse_pca"]["pca_comps"])
pca_mean = np.array(pitchfork_dict["custom_objects"]["inverse_pca"]["pca_mean"])
weights = pitchfork_dict["custom_objects"]["WMSE"]["weights"]

inp_mean = np.array(list((pitchfork_dict["data_scaling"]["inp_mean"].values())))
inp_std = np.array(list((pitchfork_dict["data_scaling"]["inp_std"].values())))

classical_out_mean = np.array(list((pitchfork_dict["data_scaling"]["classical_out_mean"].values())))
classica_out_std = np.array(list((pitchfork_dict["data_scaling"]["classical_out_std"].values())))

astero_out_mean = np.array(list((pitchfork_dict["data_scaling"]["astero_out_mean"].values())))
astero_out_std = np.array(list((pitchfork_dict["data_scaling"]["astero_out_std"].values())))

# 3) Load in emulator with custom objects
Loading in the emulator from keras .model file using custom objects from dict